# Instagram Capstone Image & Text Pre-Processing

In [1]:
import numpy as np
import pandas as pd

from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.xception import Xception
from keras.models import Model
from PIL import Image

Using TensorFlow backend.


In [2]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [4]:
# extract features from each photo in the directory
def extract_features(directory):
    # load the model
    model = VGG16()
    # re-structure the model
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    # summarize
    print(model.summary())
    # extract features from each photo
    features = dict()
    for name in listdir(directory):
        # load an image from file
        filename = directory + '/' + name
        image = load_img(filename, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)
        # get features
        feature = model.predict(image, verbose=0)
        # get image id
        image_id = name.split('.')[0]
        # store feature
        features[image_id] = feature
    return features

# extract features from all images
directory = './insta/insta_images/instabang_images/'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features_instabang.pkl', 'wb'))

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

## Text Pre-processing

Loading file with all image decriptions

In [4]:
# load doc into memory
filename = './instabang/instabang_text/instabang_index.txt'
filename2 = './flicker8k-dataset/Flickr8k_text/Flickr8k.token.txt'

# load descriptions
doc = load_doc(filename)
doc2 = load_doc(filename2)

Iterating through the photo descriptions doc and creating a dictionary of file names as keys, and descriptors as values. 

In [235]:
finding = pd.read_csv('./instabang/instabang_text/instabang_index.txt', sep='\\', header=None)

In [238]:
tes = []
for row in doc.split('\n'):
    r = row.split('\\')
    if len(r[0]) >= 30:
        print(r[0])

2019-10-06_12-05-12_UTC_10.jpg
2020-01-09_01-35-42_UTC_10.jpg
2020-01-21_19-08-08_UTC_10.jpg
2019-12-29_17-42-57_UTC_10.jpg
2019-12-31_17-44-52_UTC_10.jpg
2019-05-03_16-56-56_UTC_10.jpg
2020-01-05_16-34-27_UTC_10.jpg
2017-02-27_21-25-25_UTC_10.jpg
2018-06-03_00-29-22_UTC_10.jpg
2018-10-18_16-36-22_UTC_10.jpg
2019-01-02_07-29-40_UTC_10.jpg
2020-01-10_18-31-01_UTC_10.jpg
2020-02-11_18-55-55_UTC_10.jpg
2020-02-20_20-07-48_UTC_10.jpg
2017-08-28_01-58-30_UTC_10.jpg
2017-11-06_18-03-11_UTC_10.jpg
2018-03-21_21-27-41_UTC_10.jpg
2018-06-12_00-44-31_UTC_10.jpg
2018-11-22_15-46-25_UTC_10.jpg
2019-01-05_16-46-07_UTC_10.jpg
2019-01-14_23-30-41_UTC_10.jpg
2019-02-26_14-56-57_UTC_10.jpg
2019-02-27_16-17-42_UTC_10.jpg
2019-03-01_02-17-18_UTC_10.jpg
2019-03-04_17-38-26_UTC_10.jpg
2019-03-08_21-28-03_UTC_10.jpg
2019-04-14_21-29-11_UTC_10.jpg
2019-04-25_21-17-56_UTC_10.jpg
2019-11-25_04-21-46_UTC_10.jpg
2020-02-07_22-47-12_UTC_10.jpg
2020-02-10_21-28-56_UTC_10.jpg
2018-12-31_18-10-15_UTC_10.jpg
2019-05-

In [5]:
def load_descriptions(doc):
    mapping = dict()
    for line in doc.split('\n'):
        tokens = line.split('\\')
        if len(line) < 2:
            continue
        image_id, image_desc = tokens[0], tokens[1:]
        image_id = image_id.split('.')[0]
        if image_id not in mapping:
            mapping[image_id] = list()
        mapping[image_id].append(image_desc)
    return mapping

descriptions_insta = load_descriptions(doc)
print('Loaded:', len(descriptions_insta))

Loaded: 10961


In [27]:
def load_descriptions(doc):
    mapping = dict()
    # process lines
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        if len(line) < 2:
            continue
        # take the first token as the image id, the rest as the description
        image_id, image_desc = tokens[0], tokens[1:]
        # remove filetype from filename
        image_id = image_id.split('.')[0]
        # convert description tokens back to string
        image_desc = ' '.join(image_desc)
        # create the list if needed
        if image_id not in mapping:
            mapping[image_id] = list()
        # store description
        mapping[image_id].append(image_desc)
    return mapping

# parse descriptions
descriptions = load_descriptions(doc2)
print('Loaded: %d ' % len(descriptions))

Loaded: 8092 


### Cleaning the text
The captions are already tokenized (filenames included). Reduction in size of the vocabulary of words needs to be done with the following methods;

- All letters must be lowercase, to avoid duplication of capitalised and lower cased words. 
- Remove all punctuation. 
- Remove all words with character length of 1. 
- Remove all words that are alphanumeric. 

In [28]:
import string

def clean_descriptions(descriptions):
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            # tokenize
            desc = desc.split()
            # convert to lower case
            desc = [word.lower() for word in desc]
            # remove punctuation from each token
            desc = [w.translate(table) for w in desc]
            # remove hanging 's' and 'a'
            desc = [word for word in desc if len(word)>1]
            # remove tokens with numbers in them
            desc = [word for word in desc if word.isalpha()]
            # store as string
            desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [6]:
import string

def clean_descriptions_insta(descriptions):
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[0][i]
            # tokenize
            desc = desc.split()
            # convert to lower case
            desc = [word.lower() for word in desc]
            # remove punctuation from each token
            desc = [w.translate(table) for w in desc]
            # remove hanging 's' and 'a'
            desc = [word for word in desc if len(word)>1]
            # remove tokens with numbers in them
            desc = [word for word in desc if word.isalpha()]
            # store as string
            desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions_insta(descriptions_insta)

Need to convert the captions into a vocabulary of words. Iterate through the descriptions dictionary keys, and split each caption up by individual words. 

Printing the length of the full vocabulary of the Flikr captions presents 8,763 words, and 20,477 for Instagram.

In [9]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
    # build a list of all description strings
    all_desc = set()
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
    return all_desc

# summarize vocabulary
insta_vocabulary = to_vocabulary(descriptions_insta)
print('Vocabulary Size: %d' % len(insta_vocabulary))

Vocabulary Size: 20477


In [29]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
    # build a list of all description strings
    all_desc = set()
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
    return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))

Vocabulary Size: 8763


Then save a list of every caption and corresponding file name per line, as a .txt file. 

In [35]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

# save descriptions
save_descriptions(descriptions, 'descriptions.txt')

In [34]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc[0])
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

# save descriptions
save_descriptions(descriptions_insta, 'insta_descriptions.txt')